In [ ]:
import pandas as pd 
from sqlalchemy import create_engine 
import numpy as np

### Extract the data

In [ ]:
#Extracts the first sheet "2013-2020 Police Killings" from the xls spreadsheet

data_file = "data/OverallPoliceKillingData.xlsx"
police_data_df = pd.read_excel(data_file, sheet_name="2013-2020 Police Killings")
police_data_df.head()

In [ ]:
dod_equipment_file = "data/dod_equipment.csv"
dod_equipment_df = pd.read_csv(dod_equipment_file)
dod_equipment_df.head()

In [ ]:
police_contracts_file = "data/police_contracts_dataset.csv"
police_contracts_df = pd.read_csv(police_contracts_file)
police_contracts_df.head()

### Edit columns

In [5]:
##Rename columns so they can easily be queried
police_data_df.columns = ['victim_name','victim_age','victim_gender','victim_race','url_image_of_victim','date_of_incident',
                          'street_address_of_incident','city','state','zipcode','county','agency_responsible_for_death',
                          'cause_of_death','description_of_the_circumstances','official_disposition_of_death','criminal_charges',
                          'link_to_news_article_or_photo','symptoms_of_mental_illness','armed_or_unarmed','alleged_weapon',
                          'alleged_threat_level','fleeing','body_camera','wapo_id','off_duty_killing','suburban_rural_classification',
                         'id']
police_data_df.head()

,victim_name,victim_age,victim_gender,victim_race,url_image_of_victim,date_of_incident,street_address_of_incident,city,state,zipcode,...,symptoms_of_mental_illness,armed_or_unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,wapo_id,off_duty_killing,suburban_rural_classification,id
0,Chazz Hailey,22,Male,Pacific Islander,NaN,2020-06-23,1900 Pebblebrook Lane,Sherman,TX,75092.0,...,No,Allegedly Armed,knife,attack,NaN,NaN,NaN,NaN,NaN,NaN
1,Kevin Pulido,43,Male,Hispanic,NaN,2020-06-23,Mesa Avenue and Bragdon Avenue,Pueblo,CO,81004.0,...,No,Allegedly Armed,gun,attack,NaN,No,5943.0,NaN,NaN,NaN
2,Matthew L. Fox,40,Male,Unknown race,NaN,2020-06-23,Blind Nick Drive,Wasilla,AK,99654.0,...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Name withheld by police,43,Male,Black,NaN,2020-06-22,W Independence St & N Waco St,Giddings,TX,78942.0,...,No,Allegedly Armed,knife,attack,Not fleeing,No,5945.0,NaN,NaN,NaN
4,Name withheld by police,NaN,Male,Unknown race,NaN,2020-06-22,East 35th Place and Sable Boulevard,Aurora,CO,80011.0,...,No,Allegedly Armed,gun,attack,Not fleeing,Yes,5938.0,NaN,NaN,NaN


In [6]:
##Update to include only columns needed for data analysis

police_cols = ["victim_name","victim_age","victim_gender","victim_race","url_image_of_victim","date_of_incident","street_address_of_incident",
               "city","state","zipcode","county","agency_responsible_for_death","cause_of_death","description_of_the_circumstances",
               "official_disposition_of_death","criminal_charges","link_to_news_article_or_photo","symptoms_of_mental_illness","armed_or_unarmed",
               "alleged_weapon","alleged_threat_level","fleeing","body_camera","off_duty_killing","suburban_rural_classification"]
police_df_cleaned = police_data_df[police_cols].copy()

In [7]:
police_df_cleaned.head()

,victim_name,victim_age,victim_gender,victim_race,url_image_of_victim,date_of_incident,street_address_of_incident,city,state,zipcode,...,criminal_charges,link_to_news_article_or_photo,symptoms_of_mental_illness,armed_or_unarmed,alleged_weapon,alleged_threat_level,fleeing,body_camera,off_duty_killing,suburban_rural_classification
0,Chazz Hailey,22,Male,Pacific Islander,NaN,2020-06-23,1900 Pebblebrook Lane,Sherman,TX,75092.0,...,No known charges,https://www.heralddemocrat.com/news/20200623/t...,No,Allegedly Armed,knife,attack,NaN,NaN,NaN,NaN
1,Kevin Pulido,43,Male,Hispanic,NaN,2020-06-23,Mesa Avenue and Bragdon Avenue,Pueblo,CO,81004.0,...,No known charges,https://www.chieftain.com/news/20200623/1-dead...,No,Allegedly Armed,gun,attack,NaN,No,NaN,NaN
2,Matthew L. Fox,40,Male,Unknown race,NaN,2020-06-23,Blind Nick Drive,Wasilla,AK,99654.0,...,No known charges,https://www.ktva.com/story/42282005/man-dies-i...,No,Allegedly Armed,gun,NaN,NaN,NaN,NaN,NaN
3,Name withheld by police,43,Male,Black,NaN,2020-06-22,W Independence St & N Waco St,Giddings,TX,78942.0,...,No known charges,https://www.kxan.com/news/crime/several-people...,No,Allegedly Armed,knife,attack,Not fleeing,No,NaN,NaN
4,Name withheld by police,NaN,Male,Unknown race,NaN,2020-06-22,East 35th Place and Sable Boulevard,Aurora,CO,80011.0,...,No known charges,https://www.denverpost.com/2020/06/22/aurora-p...,No,Allegedly Armed,gun,attack,Not fleeing,Yes,NaN,NaN


In [8]:
##Rename columns so they can easily be queried
dod_equipment_df.columns = ['state','station_name','nsn','item_name','quantity','ui','acquisition_value','demil_code','demil_ic','ship_date','state']
dod_equipment_df.head(10)

,state,station_name,nsn,item_name,quantity,ui,acquisition_value,demil_code,demil_ic,ship_date,state
0,AL,ABBEVILLE POLICE DEPT,6115-01-285-3012,"GENERATOR SET,DIESEL ENGINE",2,Each,9922.25,A,7.0,00:00.0,Alabama
1,AL,ABBEVILLE POLICE DEPT,6230-01-439-3732,"FLOODLIGHT SET,ELECTRIC",1,Each,12000.00,A,1.0,00:00.0,Alabama
2,AL,ABBEVILLE POLICE DEPT,7125-01-466-0952,"CABINET,STORAGE",4,Each,860.74,A,1.0,00:00.0,Alabama
3,AL,ABBEVILLE POLICE DEPT,2530-01-558-2138,"WHEEL,PNEUMATIC TIRE",1,Assembly,2015.00,A,1.0,00:00.0,Alabama
4,AL,ABBEVILLE POLICE DEPT,2320-01-447-3892,"TRUCK,WRECKER",1,Each,331680.00,Q,6.0,00:00.0,Alabama
5,AL,ABBEVILLE POLICE DEPT,5180-01-628-2375,"TOOL KIT,AIRCRAFT MAINTENANCE",3,Kit,1298.00,A,1.0,00:00.0,Alabama
6,AL,ABBEVILLE POLICE DEPT,1240-01-411-1265,"SIGHT,REFLEX",9,Each,335.00,D,1.0,00:00.0,Alabama
7,AL,ABBEVILLE POLICE DEPT,4010-00-473-6166,"CHAIN ASSEMBLY,SINGLE LEG",10,Each,160.18,A,1.0,00:00.0,Alabama
8,AL,ABBEVILLE POLICE DEPT,5855-DS-THR-MIMG,THERMAL IMAGINING EQUIPMENT,1,Each,0.00,D,NaN,00:00.0,Alabama
9,AL,ABBEVILLE POLICE DEPT,1005-01-587-7175,"MOUNT,RIFLE",10,Each,1647.00,D,1.0,00:00.0,Alabama


In [17]:
dod_cols = ["state","station_name","nsn","item_name","quantity","ui","acquisition_value","demil_code","demil_ic","ship_date"]
dod_equipment_cleaned = dod_equipment_df[dod_cols].copy()

In [18]:
dod_equipment_cleaned.head()

,state,state,station_name,nsn,item_name,quantity,ui,acquisition_value,demil_code,demil_ic,ship_date
0,AL,Alabama,ABBEVILLE POLICE DEPT,6115-01-285-3012,"GENERATOR SET,DIESEL ENGINE",2,Each,9922.25,A,7.0,00:00.0
1,AL,Alabama,ABBEVILLE POLICE DEPT,6230-01-439-3732,"FLOODLIGHT SET,ELECTRIC",1,Each,12000.00,A,1.0,00:00.0
2,AL,Alabama,ABBEVILLE POLICE DEPT,7125-01-466-0952,"CABINET,STORAGE",4,Each,860.74,A,1.0,00:00.0
3,AL,Alabama,ABBEVILLE POLICE DEPT,2530-01-558-2138,"WHEEL,PNEUMATIC TIRE",1,Assembly,2015.00,A,1.0,00:00.0
4,AL,Alabama,ABBEVILLE POLICE DEPT,2320-01-447-3892,"TRUCK,WRECKER",1,Each,331680.00,Q,6.0,00:00.0


In [11]:
##Rename columns so they can easily be queried
police_contracts_df.columns = ['city','expiration','section','policy_language','category','specific_impact_of_policy','id']
police_contracts_df.head(10)

,city,expiration,section,policy_language,category,specific_impact_of_policy,id
0,Albuquerque,7/16/2015,18.1.4,Derogatory material may be purged within twelv...,Erases misconduct records,Erases records at department head's discretion,43
1,Albuquerque,7/16/2015,18.1.5,"""Human Resources Department files are a perman...",Erases misconduct records,Erases records if charges are dropped,46
2,Albuquerque,7/16/2015,20.1.12,"When available, before an administrative inves...",Gives officers unfair access to info,Gives officers access to the names of complain...,180
3,Albuquerque,7/16/2015,20.1.3,"The name of the charging officer, complaintant...",Gives officers unfair access to info,Gives officers access to the names of complain...,174
4,Albuquerque,7/16/2015,20.1.9,if the questioning is mechanically or stenogra...,Gives officers unfair access to info,Gives officers immediate access to the recorde...,516
5,Albuquerque,7/16/2015,20.1.10,Any information released to the Police Oversig...,Limits oversight/discipline,Prevents the identity of the officer or inform...,404
6,Albuquerque,7/16/2015,23.1.1,Should a police officer be sued in a civil act...,Requires city pay for misconduct,City pays costs of misconduct settlements and ...,337
7,Albuquerque,7/16/2015,20.1.5.1,The interrogation shall be completed as soon a...,Restricts/delays interrogations,Limits interrogations to two 2-hour sessions w...,250
8,Albuquerque,7/16/2015,20.1.6,"Only two interrogators, the involved officer a...",Restricts/delays interrogations,Limits the number of interrogators to two,261
9,Albuquerque,7/16/2015,20.1.6,The officer shall not be subjected to any offe...,Restricts/delays interrogations,Limits the type of language and incentives use...,275


In [19]:
pc_cols = ["city","policy_language","category","specific_impact_of_policy"]
police_contracts_cleaned = police_contracts_df[pc_cols].copy()

In [20]:
police_contracts_cleaned.head()

,city,policy_language,category,specific_impact_of_policy
0,Albuquerque,Derogatory material may be purged within twelv...,Erases misconduct records,Erases records at department head's discretion
1,Albuquerque,"""Human Resources Department files are a perman...",Erases misconduct records,Erases records if charges are dropped
2,Albuquerque,"When available, before an administrative inves...",Gives officers unfair access to info,Gives officers access to the names of complain...
3,Albuquerque,"The name of the charging officer, complaintant...",Gives officers unfair access to info,Gives officers access to the names of complain...
4,Albuquerque,if the questioning is mechanically or stenogra...,Gives officers unfair access to info,Gives officers immediate access to the recorde...


### Update data types

In [21]:
##Get data types
police_df_cleaned.dtypes

victim_name                                 object
victim_age                                  object
victim_gender                               object
victim_race                                 object
url_image_of_victim                         object
date_of_incident                    datetime64[ns]
street_address_of_incident                  object
city                                        object
state                                       object
zipcode                                    float64
county                                      object
agency_responsible_for_death                object
cause_of_death                              object
description_of_the_circumstances            object
official_disposition_of_death               object
criminal_charges                            object
link_to_news_article_or_photo               object
symptoms_of_mental_illness                  object
armed_or_unarmed                            object
alleged_weapon                 

In [22]:
dod_equipment_cleaned.dtypes

state                 object
state                 object
station_name          object
nsn                   object
item_name             object
quantity               int64
ui                    object
acquisition_value    float64
demil_code            object
demil_ic             float64
ship_date             object
dtype: object

In [23]:
police_contracts_cleaned.dtypes

city                         object
policy_language              object
category                     object
specific_impact_of_policy    object
dtype: object

In [ ]:
#Update zip code datatypes where needed
police_df_cleaned.zipcode = police_df_cleaned.zipcode.astype('Int64')

police_df_cleaned.dtypes

In [ ]:
police_df_cleaned.head(10)

In [ ]:
##Get unique values for victim_age.  To understand what values are being captured
police_df_cleaned.victim_age.unique()

In [ ]:
police_df_cleaned['victim_age'].replace(['Unknown','40s'],[-1,-1],inplace=True) 
police_df_cleaned.victim_age.unique()

In [ ]:
police_df_cleaned['victim_age'] = police_df_cleaned['victim_age'].astype('Int64')
police_df_cleaned.dtypes

In [ ]:
police_df_cleaned.head(10)

In [ ]:
police_df_cleaned.victim_age.unique()

In [ ]:
police_df_cleaned['victim_age'].replace([-1],[np.nan],inplace=True) 
police_df_cleaned.victim_age.unique()

In [ ]:
police_df_cleaned['victim_age'] = police_df_cleaned['victim_age'].astype('Int64')
police_df_cleaned.dtypes

### Create database connection

In [ ]:
connection_string = "postgres:password@localhost:5432/us_incarceration_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load dataframe into database

In [ ]:
police_df_cleaned.to_sql(name='us_police_killing_data', con=engine, if_exists='append', index=True)